In [4]:
import numpy as np
import pandas as pd
import asyncio
#import aiohttp
import json
import csv

In [5]:
import requests
import json
import time
from datetime import datetime
from datetime import date
import numpy as np
import pandas as pd

def get_markets(api = "cryptowatch", active = True):
    """get markets available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                exchanges = {item['exchange'] for item in r if item['active'] is True}
            else:
                exchanges = {item['exchange'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
            

    return exchanges

def get_pairs(api = "cryptowatch", active = True):
    """get pairs available in API"""
    if api == "cryptowatch":
        try:
            r = requests.get("https://api.cryptowat.ch/markets").json()['result']
            if (active):
                pairs = {item['pair'] for item in r if item['active'] is True}
            else:
                pairs = {item['pair'] for item in r}
        except:
            print("Error: disfunctional API from Cryptowatch")
    return pairs



In [203]:
p = get_pairs()

def list_pairs(pairs=get_pairs(), p1='btc'):
    pairs = list(pairs)
    L = []
    for pair in pairs:
        if p1 == pair[:len(p1)]:
            L.append(pair)
    return L
#get_markets()

In [205]:
a = list_pairs()
print(a)

['btceur', 'btckrw', 'btcpeth', 'btcjpy-weekly-futures', 'btcgbp', 'btczar', 'btctusd', 'btcidr', 'btcusd-weekly-futures', 'btcfxjpy', 'btcngn', 'btcjpy-quarterly-futures', 'btccad', 'btcusd-biweekly-futures', 'btcmyr', 'btchkd', 'btcsgd', 'btcjpy', 'btcusdt', 'btcpln', 'btcusd-biquarterly-futures', 'btcdxmr', 'btcaud', 'btcusd-perpetual-futures', 'btcinr', 'btcdbtc', 'btcpusdt', 'btcusd-monthly-futures', 'btcdai', 'btcusd', 'btcjpy-biweekly-futures', 'btcphp', 'btcbtc-weekly-futures', 'btcrur', 'btcpbtc', 'btccny', 'btcusd-quarterly-futures']


In [167]:
def get_ohlc(market, pair, period = 1440, api = "cryptowatch", start = "", end = "", local_timezone = True):
    """get OHLC prices available in API"""
    period *= 60
    period = str(period)
    params = []
    url = "https://api.cryptowat.ch/markets/" + market + "/" + pair + "/ohlc"
#    print(url)
    if (end == ""):
        before = ""
    else:
        before = time.mktime(time.strptime(end, "%d %m %y %H %M"))
    if (start == ""):
        after = start
    else:
        after = time.mktime(time.strptime(start, "%d %m %y %H %M"))
    if (not before or not after):
        prms = [('before', before), ('after', after)]
    else:
        prms = {'before': int(before) , 'after': int(after) }

    
    if api == "cryptowatch":
        try:
            r = requests.get(url, params=prms).json()['result'][period]
            if (local_timezone):            
                a = pd.DataFrame(r, columns = ['date(local)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]] - time.timezone, unit = 's')
                a = a.set_index('date(local)')
 
                return a
            else:
                a = pd.DataFrame(r, columns = ['date(utc)','open','high','low','close','volume','neg'])
                a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')
                a = a.set_index('date(utc)')

                return a               
                

        except:
            print("Error: disfunctional API from Cryptowatch")
            
#columns = ['period','open','high','low','close','volume','neg']    

In [154]:
#a = get_ohlc("bitfinex", "btcusd", period = 1, start = time.time()-24*60*60, end = time.time())

s1 = "14 07 18 00 00"
e1 = "15 07 18 00 00"
#print(time.timezone)
a = get_ohlc("bitfinex", "btcusd", period = 240, start = s1, end = e1)

a.loc['2018-07-14 01:00:00']



https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc


open      6.219700e+03
high      6.277600e+03
low       6.196000e+03
close     6.208400e+03
volume    1.454638e+03
neg       9.076131e+06
Name: 2018-07-14 01:00:00, dtype: float64

In [146]:
a = get_ohlc("bitfinex", "btcusd", period = 240, start = s1, end = e1)

a = a.set_index('date(local)')
print(a.index[0])
print(str(a.index[0]))
a.loc['2018-07-13 21:00:00']
a[3:5]

https://api.cryptowat.ch/markets/bitfinex/btcusd/ohlc
2018-07-13 21:00:00
2018-07-13 21:00:00


,open,high,low,close,volume,neg
date(local),,,,,,
2018-07-14 09:00:00,6221.3115,6242.4,6179.2,6230.9,1382.91420,8586973
2018-07-14 13:00:00,6231.0000,6251.2,6221.1,6242.7,953.43585,5947307


In [156]:
def get_daily_price(asset="btc", api="coinmetrics", data_type="price(usd)", start=0, end=time.time()):
    if (api == "coinmetrics"):
        try:
            url =  "https://coinmetrics.io/api/v1/get_asset_data_for_time_range/"+asset+"/"+data_type
            url = url + "/"+str(int(start)) +"/"+str(int(end))
            r = requests.get(url).json()['result']
            #print(type(r))
            #print(r)
            a = pd.DataFrame(r, columns = ['date(utc)',str(asset)+":"+str(data_type)])
            print(url)
            a[a.columns[0]] = pd.to_datetime(a[a.columns[0]], unit = 's')

            return a
        except:
            print("Error: disfunctional API from Coinmetrics")            
            
    else:
        return 0



In [158]:
a = get_daily_price()
print(time.time())
a.tail()


https://coinmetrics.io/api/v1/get_asset_data_for_time_range/btc/price(usd)/0/1531949048
1531949052.3189657


,date(utc),btc:price(usd)
1902,2018-07-13,6235.03
1903,2018-07-14,6247.50
1904,2018-07-15,6272.70
1905,2018-07-16,6357.01
1906,2018-07-17,6739.65


# Terça-Feira

- finalizar get_ohlc() (from datetime to timestamp)

- atualizar get_pairs() colocando um parâmetro active = True para retornar só pares ativos

```Python 
pairs = {item['pair'] for item in r if item['active'] is True} ```

- O mesmo para mercados

